# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port elizabeth,-33.9180,25.5701,10.17,88,0,2.87,ZA,1723000467
1,1,udachny,66.4167,112.4000,14.56,48,46,1.20,RU,1723000468
2,2,kapuskasing,49.4169,-82.4331,10.91,79,34,0.82,CA,1723000202
3,3,port alfred,-33.5906,26.8910,13.10,68,0,3.62,ZA,1723000471
4,4,vanimo,-2.6741,141.3028,28.42,78,98,3.80,PG,1723000472


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_data_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 580,
    size = "Humidity",
    scale = 1,
    color = "City")

# Display the map
city_data_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
Ideal_City = city_data_df.loc[(city_data_df["Max Temp"] < 34) & (city_data_df["Max Temp"] > 19) & (city_data_df["Cloudiness"] < 75) & (city_data_df["Wind Speed"] < 10) & (city_data_df["Humidity"] < 65)]


# Drop any rows with null values
Ideal_City = Ideal_City.dropna()

# Display sample data
Ideal_City.head(20)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,7,adrar,20.5022,-10.0711,32.17,34,14,5.12,MR,1723000475
11,11,buka,40.8108,69.1986,28.52,21,0,4.59,UZ,1723000479
20,20,yellowknife,62.4560,-114.3525,19.67,49,9,4.02,CA,1723000490
51,51,undurkhaan,47.3194,110.6556,23.47,51,46,1.53,MN,1723000525
65,65,alaghsas,17.0187,8.0168,27.98,61,27,6.13,NE,1723000541
82,82,zouerate,22.7187,-12.4521,30.84,21,4,2.91,MR,1723000560
85,85,vilhena,-12.7406,-60.1458,20.57,42,4,1.89,BR,1723000372
91,91,labasa,-16.4167,179.3833,25.49,53,5,5.13,FJ,1723000570
130,130,dalandzadgad,43.5708,104.4250,29.46,30,42,8.98,MN,1723000614
135,135,baotou,40.6522,109.8222,26.44,48,22,3.63,CN,1723000619


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = Ideal_City[["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
7,adrar,MR,20.5022,-10.0711,34,
11,buka,UZ,40.8108,69.1986,21,
20,yellowknife,CA,62.4560,-114.3525,49,
51,undurkhaan,MN,47.3194,110.6556,51,
65,alaghsas,NE,17.0187,8.0168,61,
...,...,...,...,...,...,...
564,cacu,BR,-18.5567,-51.1308,38,
574,staryy krym,UA,45.0333,35.1000,57,
585,gaozhou,CN,21.9392,110.8461,61,
587,los villares,ES,37.3482,-3.3314,30,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories":"accommodation.hotel", "apiKey":geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]


    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
adrar - nearest hotel: No hotel found
buka - nearest hotel: No hotel found
yellowknife - nearest hotel: No hotel found
undurkhaan - nearest hotel: Khanburged Hotel
alaghsas - nearest hotel: Hôtel Zagado
zouerate - nearest hotel: فندق تيرس زمور
vilhena - nearest hotel: No hotel found
labasa - nearest hotel: Grand Eastern Hotel
dalandzadgad - nearest hotel: Хан Уул буудал
baotou - nearest hotel: 包头宾馆
ivolginsk - nearest hotel: No hotel found
crane - nearest hotel: No hotel found
alice springs - nearest hotel: Aurora Alice Springs
weiser - nearest hotel: State Street Motel
whitehorse - nearest hotel: Town & Mountain Hotel
uruzgan - nearest hotel: No hotel found
kidal - nearest hotel: No hotel found
viqueque - nearest hotel: Hotel Wailakurini
ghadamis - nearest hotel: فندق عين الفارس
karratha - nearest hotel: Karratha International Hotel
qarqaraly - nearest hotel: Каркаралинск
berezovyy - nearest hotel: No hotel found
metro - nearest hotel: Hotel Nuban
qui nhon - near

,City,Country,Lat,Lng,Humidity,Hotel Name
7,adrar,MR,20.5022,-10.0711,34,No hotel found
11,buka,UZ,40.8108,69.1986,21,No hotel found
20,yellowknife,CA,62.4560,-114.3525,49,No hotel found
51,undurkhaan,MN,47.3194,110.6556,51,Khanburged Hotel
65,alaghsas,NE,17.0187,8.0168,61,Hôtel Zagado
...,...,...,...,...,...,...
564,cacu,BR,-18.5567,-51.1308,38,No hotel found
574,staryy krym,UA,45.0333,35.1000,57,No hotel found
585,gaozhou,CN,21.9392,110.8461,61,城市便捷酒店
587,los villares,ES,37.3482,-3.3314,30,Hotel Cerrilo San Marcos


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_df_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 520,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols=["Lat","Lng","City","Humidity","Hotel Name","Country"])


# Display the map
hotel_df_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)